# Determine support assignment gaps

In [1]:
import pandas as pd
%matplotlib inline

from entities import CoachingPracticeFinance, ExpenseLineItem, HoursLineItem

practice = CoachingPracticeFinance.load("../practice.json")

In [2]:
period_start = pd.to_datetime("2022-11-01")
period_end = pd.to_datetime("2023-10-31")
report_date_range = pd.bdate_range(period_start, period_end, freq="C", holidays=practice.statutory_holiday_list)
report_frame = pd.DataFrame(report_date_range, columns=["date"])
report_frame

,date
0,2022-11-01
1,2022-11-02
2,2022-11-03
3,2022-11-04
4,2022-11-07
...,...
246,2023-10-25
247,2023-10-26
248,2023-10-27
249,2023-10-30


In [7]:
assignment_coverage = []
columns = ['date'] + [a.code for a in practice.support_areas]
for date in report_date_range:
    row = [date]
    for a in practice.support_areas:
        covered = False
        for aa in practice.area_assignments:
            if aa.support_area_code == a.code:
                start = pd.to_datetime(aa.start_date)
                end = pd.to_datetime(aa.end_date)
                if start <= date <= end:
                    covered = True
        row.append(covered)
    assignment_coverage.append(row)
assignments_frame = pd.DataFrame(assignment_coverage, columns=columns)
assignments_frame

,date,homeX,CB,Dojo
0,2022-11-01,True,False,True
1,2022-11-02,True,False,True
2,2022-11-03,True,False,True
3,2022-11-04,True,False,True
4,2022-11-07,True,False,True
...,...,...,...,...
246,2023-10-25,False,True,False
247,2023-10-26,False,True,False
248,2023-10-27,False,True,False
249,2023-10-30,False,True,False


In [8]:
df = report_frame.merge(assignments_frame, how="outer", on="date")
df

,date,homeX,CB,Dojo
0,2022-11-01,True,False,True
1,2022-11-02,True,False,True
2,2022-11-03,True,False,True
3,2022-11-04,True,False,True
4,2022-11-07,True,False,True
...,...,...,...,...
246,2023-10-25,False,True,False
247,2023-10-26,False,True,False
248,2023-10-27,False,True,False
249,2023-10-30,False,True,False


In [9]:
for a in practice.support_areas:
    df[f"{a.code}_group"] = (df[a.code].diff() != 0).cumsum()
df

,date,homeX,CB,Dojo,homeX_group,CB_group,Dojo_group
0,2022-11-01,True,False,True,1,1,1
1,2022-11-02,True,False,True,1,1,1
2,2022-11-03,True,False,True,1,1,1
3,2022-11-04,True,False,True,1,1,1
4,2022-11-07,True,False,True,1,1,1
...,...,...,...,...,...,...,...
246,2023-10-25,False,True,False,2,2,2
247,2023-10-26,False,True,False,2,2,2
248,2023-10-27,False,True,False,2,2,2
249,2023-10-30,False,True,False,2,2,2


In [10]:
from tabulate import tabulate

lines = []
for a in practice.support_areas:
    unstaffed = df[df[a.code] == False]
    report = unstaffed.groupby(f"{a.code}_group")['date'].agg(['min', 'max'])
    report = report.reset_index(drop=True)
    if report.count()[0] > 0:
        lines.append("")
        lines.append(f"{a.code} is Unstaffed")
        lines.append("")
        report['min'] = report['min'].dt.strftime('%Y-%m-%d')
        report['max'] = report['max'].dt.strftime('%Y-%m-%d')
        report.columns = ['Start Date', 'End Date']
        lines.append(tabulate(report, showindex=False))
print("\n".join(lines))


homeX is Unstaffed

----------  ----------
2023-10-02  2023-10-31
----------  ----------

CB is Unstaffed

----------  ----------
2022-11-01  2023-04-28
----------  ----------

Dojo is Unstaffed

----------  ----------
2023-10-02  2023-10-31
----------  ----------
